<a href="https://colab.research.google.com/github/oliviajakli/oliviajakli/blob/main/1_SaB_firstImageRun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Authorizations

In [ ]:
# Imports used in this notebook.
import ee
import os
import folium
import sys
import time
import shutil
import pandas as pd
import seaborn as sea
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from ee import batch
from matplotlib import cm
from google.colab import drive

# Must authenticate your EE account before use of the package.
ee.Authenticate()

ee.Initialize(project='ksu-skytruth-vpca-automation')
# Try adding ,opt_url='https://earthengine-highvolume.googleapis.com')

# This is how we can access our drive to get the correlation product.
drive.mount('/content/gdrive')

# Set the IRL AOI.

In [ ]:
#Sandusky Bay, Ohio
siteName = 'Sandusky Bay'
#Must be 3 characters long
siteName_2char = 'SaB'

#Define coordinates
upperLeft = [-83.20, 41.55]
lowerRight = [-82.575, 41.36]

# Define a region of interest to filter our collection.
roi = ee.Geometry.Rectangle(upperLeft[0], upperLeft[1], lowerRight[0], lowerRight[1]) # vector data type describing a rectangular polygon

# Select our Landsat 8 imagery


In [ ]:
# List of dates.
date_list = [
ee.Date('2023-09-01'),
ee.Date('2023-08-22'), # cloudiest
ee.Date('2023-07-18'), # 2nd most cloudy
ee.Date('2023-06-18'), # could replace with '2023-06-03'
ee.Date('2023-05-09'), # could replace with '2023-05-29'
ee.Date('2022-09-01'),
ee.Date('2022-08-27'), # could replace with '2022-08-12'
ee.Date('2022-07-03'),
ee.Date('2022-06-18'), # could replace with '2022-06-28'
ee.Date('2022-05-09'),
ee.Date('2021-09-06'),
ee.Date('2021-08-22'),
ee.Date('2021-07-18'), # could replace with '2021-07-28'
ee.Date('2021-06-13'), # lots of cloud cover
ee.Date('2021-05-21') # could replace with '2021-05-14'
# ee.Date('2020-09-21'),
# ee.Date('2020-08-22'),
# ee.Date('2020-07-18'),
# ee.Date('2020-06-08'),
# ee.Date('2020-05-24') # thin, wispy clouds
]



im_list = []

# Start Standard code for processing images from a collection.
for date in date_list:

  # Bring in our imagery.
  coll = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") # stack (time-series) of images
              .filterBounds(roi)
              .filterDate(date, date.advance(1,'day')))

  # Clip our input image to our region of interest.
  # Set the name of the original image to our new image.
  water = coll.mosaic().clip(roi).set({'name':coll.first().get('system:index')})

  # Add the newly created image to the empty list created above.
  im_list += [water]

# Make an EE Collection out of the list we created.
im_list = ee.ImageCollection(im_list)

# Begin analysis

In [ ]:
# Name the parent folder where we're writing our output files.
output_folder = '/content/gdrive/My Drive/test_SaB'

# Define the Sentinel-2 bands to extract
non_ee_band_list = ['B1','B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A']

# For reductions.
native_res = 10
maxP = 1e6

# Array of wavelength corrections for Sentinel-2 sensor.
wl = ee.Array([443.9,496.6,560.0,664.5,703.9,740.2,782.5,835.1,864.8])
# Nominal Copernicus S2A Wavelength Values
# wl = ee.Array([442.7,492.7,559.8,664.4,704.1,740.5,782.8,832.8,864.7])
# Name the band combo for outputs.
bandCombo="B1-B8A"

""" Bands and Wavelength Array to select. You can add or subtract any you'd like here and the code
    below will account for the change throughout the workflow. """

# Use this for the derivative calculation.
band_list_early_length = len(non_ee_band_list)
# We use this everywhere else after the derivative calc (original band list x2).
#band_list_length = band_list_early_length * 2
band_list_length = band_list_early_length
# Use this in a couple for loops throughout the code.
band_list_forLoops = range(band_list_length)

In [ ]:
def derivative_images(water):

  """This section is devoted to modifying the image, making it suitable for
     the Derivative Spectra function, and running the DS function."""

  # This function masks the cloudy pixels and identifies the water pixels.
  def get_water(image):

    # Water with cloud check, but note this is the not bit addition method.
    water_pixels = (image.select('QA60').eq(0).And(image.select('SCL').eq(6)))
    # water_pixels=image.select('QA60')
    # cloudBitMask=1 << 10
    # cirrusBitMask=1 << 11
    # mask = water_pixels.bitwiseAnd(cloudBitMask).eq(0).And(water_pixels.bitwiseAnd(cirrusBitMask).eq(0))
    # water_pixels = (image.select('SCL').eq(6).And(image.select('SCL').lt(7)))
    # Mask all non-water pixels in the image.
    image = image.updateMask(water_pixels)

    return image

  # Run the S2 mask on the test images.
  water_mask = get_water(water)

  # Binary reducer, which returns 1 everywhere that water hasn't been masked.
  mask_image = water_mask.reduce(ee.Reducer.anyNonZero())

  # Any pixels passing the pixel mask will now be polygonized.
  water_geom = mask_image.reduceToVectors(scale = native_res,
                                          geometry = roi,
                                          maxPixels = maxP,
                                          bestEffort = True,
                                          tileScale = 16)

  # This function will select the bands of importance and clip the raster.
  def selAndClip(masked_image,geo):

    clipped = masked_image.select(non_ee_band_list).clip(geo)

    return clipped.toDouble()

  # Run the select and clip function on the test images, converts the pixels values to double precision.
  clipped_image = selAndClip(water_mask,water_geom)

  # Add code below to divide the reflectance bands by 10^4 to scale from 0 to 1
  clipped_image = clipped_image.multiply(0.0001)

  # This function calculates the derivative bands that will also be used in the PCA analysis.
  def derivativeSpectra(img,numBands):

    drdlraster = None

    for i in range(numBands):

      # Special case of the equation is required for the first band.
      if i == 0:
        drdlraster = img.expression(
          '(secondBand - firstBand)/(secondIndex - firstIndex)',
          {
          'secondBand': img.select(i+1),
          'firstBand': img.select(i),
          'secondIndex': wl.get([i+1]),
          'firstIndex': wl.get([i])
          }).rename('image_Index' + str(i) + '_derivative')

      # Special case of the equation is required for the last band.
      elif i == (numBands-1):

        drdlraster = img.expression(
          '(secondBand - firstBand)/(secondIndex - firstIndex)',
          {
          'secondBand': img.select(i),
          'firstBand': img.select(i-1),
          'secondIndex': wl.get([i]),
          'firstIndex': wl.get([i-1])
          }).rename('image_Index' + str(i) + '_derivative')

      # Apply equation for all other bands in the image.
      else:
        drdlraster = img.expression(
          '(secondBand - firstBand)/(secondIndex - firstIndex)',
          {
          'secondBand': img.select(i+1),
          'firstBand': img.select(i-1),
          'secondIndex': wl.get([i+1]),
          'firstIndex': wl.get([i-1])
          }).rename('image_Index' + str(i) + '_derivative')

      # Adds derivative bands to the input image.
      img = img.addBands(drdlraster)

    # Final image has the name of its original image reassigned.
    # This is used for file naming conventions later on.
    return img.set({'name': water.get('name')}).select(8,9,10,11,12,13,14,15,16)

  # Run the Derivative Spectra function on the test image.
  d_image = derivativeSpectra(clipped_image,band_list_early_length)
# Apply a Gaussian smoother to the image to reduce high frequency noise
#smooth_kernel hold the kernel function; select various odd values to change the kernel size.
  smooth_kernel = ee.Kernel.gaussian(radius = 3)
# apply the smoothing kernal using the ee.convolve function
  d_image = d_image.convolve(smooth_kernel)
  return d_image

In [ ]:
# Make derivative images for all of our images of interest.
der_list = im_list.map(derivative_images)

# Determine Correlation Matrix for image

In [ ]:
"""The following section is devoted to the correlation matrix
   creation process, as well as the eigen analysis."""

def pearCorr(img, starting_band_index):

  # This is distinct from the other list of band names because this is an EE object.
  # Therefore, this can utilize EE-side functions, like get().
  band_list = img.bandNames()

  # This will be filled with correlation values.
  emptyList = ee.List([])

  # Loop through band names.
  for i in band_list_forLoops:

    # Get band name based on index.
    corr_band = band_list.get(i)

    # Select the defined band from the input image.
    one = img.select([starting_band_index])

    # Select the second band of interest.
    two = img.select([corr_band])

    # Combine the two selected bands into a new image.
    cat = ee.Image.cat(one, two)

    # Compute the correlation between the bands in the newly created image.
    corr = cat.reduceRegion(
               reducer = ee.Reducer.pearsonsCorrelation(),
               geometry = roi,
               scale = native_res,
               maxPixels = maxP,
               bestEffort = True)

    # Get the correlation value out of the reduceRegion() dictionary.
    corr_value = corr.getNumber('correlation')

    # Push the new correlation value into our empty list.
    emptyList = emptyList.add(corr_value)

  return emptyList

In [ ]:
# This code will be used for making some subfolders and writing image data to said folders.

# Convert our stack of EE images to an EE List.
image_list = der_list.toList(der_list.size())
# Get the size of the list. This will be used for the list comprehension in
# the next line.
size = image_list.size().getInfo()

# Using the variables created above, get the names of the images that we are using in our analysis.
image_names = [ee.Image(image_list.get(i)).get('name').getInfo() for i in range(size)]

In [ ]:
image_names

In [ ]:
# This is where each respective image's information will be saved.
# Notice that we're using the image names we just made.
for name in image_names:
  os.mkdir(f'{output_folder}/{name}_outputs')

In [ ]:
# List our directory of folders, which we can utilize to write files.
im_folders = os.listdir(f'{output_folder}')

In [ ]:
# Append "_outputs" to the name of each image. Just a matter of file naming here.
im_folders = [name+'_outputs' for name in image_names]

In [ ]:
im_folders

In [ ]:
# Here, we export our correlation matrices and pixel counts.
def export_corr(d_image, out):
  d_image = ee.Image(d_image)

  # List comprehension to compile all of our correlation values (per image).
  # We also grab the number of pixels visible in the image, which is used for weighting.
  corr_ls = ee.Feature(None, {'correlation_list':[pearCorr(d_image, i) for i in band_list_forLoops],
                              'pixel_count': d_image.reduceRegion(ee.Reducer.count(),roi,maxPixels = maxP, scale = native_res, bestEffort = True, tileScale = 16).get('image_Index0_derivative')})

  # Name our output file.
  runtime_filename = f'6VPCA_{siteName_2char}_{d_image.get("name").getInfo()}_correlation_{bandCombo}'

  # Create our export task.
  intermission = ( batch.Export.table.toDrive(
                   collection = ee.FeatureCollection(corr_ls),
                   description = runtime_filename,
                   fileFormat = 'CSV') )

  # Initialize our task.
  batch.Task.start(intermission)

  # Monitor the task.
  i = 1
  # While running, give us an update.
  while intermission.status()['state'] in ['READY', 'RUNNING']:
    sys.stdout.write("\r" + f'Correlation export status update #{i}: ' + str(intermission.status()))
    sys.stdout.flush()
    i += 1
    #time changed from 20 double if having issues
    time.sleep(30)

  # When finished running, let us know what happened.
  else:
    print('Correlation export completed...')
    print(intermission.status())
    # Give your Drive a chance to update with changes made to your storage.
    #time changed from 20 double if having issues
    time.sleep(30)

    # Move your newly created files to the places they need to be.
    try:
      # Found here: https://stackoverflow.com/questions/51109931/moving-files-in-google-colab
      shutil.move(f'/content/gdrive/My Drive/{runtime_filename}.csv',f'{output_folder}/{out}')

    except:
      #double time if having issues
      time.sleep(60)
      shutil.move(f'/content/gdrive/My Drive/{runtime_filename}.csv',f'{output_folder}/{out}')

# Run the correlation export for every file in our image stack.
for i in range(len(im_folders)):
  export_corr(image_list.get(i),im_folders[i])

# Begin section for correlation graph

In [ ]:
# Find all of the files in our output folder.
csv_files = []
# For a description of the functionality of os.walk(), see below:
# https://www.tutorialspoint.com/python/os_walk.htm
for subdir, dirs, files in os.walk(f'{output_folder}'):
  for file in files:
    if "_correlation" in file and "Viz" not in file and 'gsheet' not in file:
      csv_files += [os.path.join(subdir,file)]

In [ ]:
csv_files

In [ ]:
# counter
i = 0

for fn in csv_files:

  # First, load our csv file into a Pandas DataFrame.
  drive_string = fn
  dataframe = pd.read_csv(drive_string, delimiter = ',', converters = {'correlation_list': eval})

  # Next, convert our dataframe to a list.
  print (dataframe)
  corr_ls = dataframe['correlation_list'].tolist()[0]

  # Initially found here: https://stackoverflow.com/questions/33282368/plotting-a-2d-heatmap-with-matplotlib
  # Create a 2D heat map using Seaborn.

  # Use this code to allow for differing number of bands processed
  labels = [f'd{non_ee_band_list[x]}' for x in range(band_list_early_length)]

  # Mask redundant info from the correlation array visualization.
  mask = np.zeros_like(corr_ls)
  mask[np.triu_indices_from(mask)] = True

  # Create matplotlib figure to place our heatmap in.
  fig = plt.figure(figsize = (20,20))
  heat = sea.heatmap(corr_ls, square = False, mask = mask, cmap = 'coolwarm', annot = True)
  heat.set_xticklabels(labels)
  heat.set_yticklabels(labels)

  for tick in heat.get_yticklabels():
    tick.set_rotation(0)

  heat.set_title(f"Correlation Matrix: {image_names[i]}")

  plt.savefig(f'{output_folder}/{im_folders[i]}/{image_names[i]}_correlationViz.jpg')
  i += 1
